In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from rich import print
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")

data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=True, sep=" ", start_row=None, end_row=None
)

# data = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=False, sep=" | ", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=",", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=False, sep=" -> ", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=" ", start_row=2, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="si", per_letter=False, sep=" ", start_row=0, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=True, sep=" ", start_row=0, end_row=None
# )

print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: {[<([<[[(<[({<<><>><()[]>}<(()())(())>)<(([]())){<()()><{}<>)}>][(((()<>)[{}{}]){
<[]<>>({}{})})]>)[(({([{}[]] | Last line: 
[(<[([([{{{<([[]<>]<<>>)[{<>[]}{{}<>}]}({(())[{}()]}[<{}<>>{<>()}])}([<[(){}]([][])><(()())

------

Convert to:

First line: ['{', '[', '<', '(', '[', '<', '[', '[', '(', '<', '[', '(', '{', '<', '<', '>', 
'<', '>', '>', '<', '(', ')', '[', ']', '>', '}', '<', '(', '(', ')', '(', ')', ')', '(', 
'(', ')', ')', '>', ')', '<', '(', '(', '[', ']', '(', ')', ')', ')', '{', '<', '(', ')', 
'(', ')', '>', '<', '{', '}', '<', '>', ')', '}', '>', ']', '[', '(', '(', '(', '(', ')', 
'<', '>', ')', '[', '{', '}', '{', '}', ']', ')', '{', '<', '[', ']', '<', '>', '>', '(', 
'{', '}', '{', '}', ')', '}', ')', ']', '>', ')', '[', '(', '(', '{', '(', '[', '{', '}', 
'[', ']', ']']

Last line: ['[', '(', '<', '[', '(', '[', '(', '[', '{', '{', '{', '<', '(', '[', '[', ']', 
'<', '>', ']', '<', '<', '>', '>', ')', '[', '{', '<', '>', '[', ']', '}', '{', '{', '}', 
'<', '>', '}', ']', '}', '(', '{', '(', '(', ')', ')', '[', '{', '}', '(', ')', ']', '}', 
'[', '<', '{', '}', '<', '>', '>', '{', '<', '>', '(', ')', '}', ']', ')', '}', '(', '[', 
'<', '[', '(', ')', '{', '}', ']', '(', '[', ']', '[', ']', ')', '>', '<', '(', '(', ')', 
'(', ')', ')']

------

In [4]:
score_map = {
    "{": 1197,
    "[": 57,
    "<": 25137,
    "(": 3,
    "}": -1197,
    "]": -57,
    ">": -25137,
    ")": -3,
}

new_lines = []
for line in data:
    digi_line = [score_map[i] for i in line]
    while True:
        list_to_pop = []
        for idx in range(len(digi_line) - 1):
            if (digi_line[idx] > 0) and (digi_line[idx] + digi_line[idx + 1] == 0):
                list_to_pop.extend([idx, idx + 1])
        digi_line = np.delete(digi_line, list_to_pop)
        if len(list_to_pop) == 0:
            break
    new_lines.append(digi_line)
    
q1 = np.sum(
    [
        -digi_line[np.argmax(digi_line < 0)] if np.any(digi_line < 0) else 0
        for digi_line in new_lines
    ]
)
    
print(f"Answer to Q1: {q1}")

Answer to Q1: 294195

In [5]:
new_score_map = {1197: 3, 57: 2, 3: 1, 25137: 4}

scores = []
for digi_line in new_lines:
    score = np.int64(0)
    if np.any(digi_line < 0):
        continue
    digi_line = np.flip([new_score_map[d] for d in digi_line])
    for s in digi_line:
        score = score*5 + s
    scores.append(score)
q2 = int(np.median(scores))
print(f"Answer to Q2: {q2}")

Answer to Q2: 3490802734